# Práctica 2 – Búsqueda en espacios de estados
## Planificación de rutas hospitalarias

Asignatura: Sistemas Inteligentes aplicados en Salud  
Autor: Darío Meneses  

Notebook de ejecución y comparación de algoritmos de búsqueda sobre el mapa del hospital.

## Objetivos de la Práctica 2

El objetivo general de esta práctica es **modelar y resolver un problema de planificación de rutas en un entorno hospitalario** mediante técnicas de **búsqueda en espacios de estados**, así como **comparar el comportamiento y rendimiento** de distintos algoritmos de búsqueda aplicados a un mismo problema.

---

### 1. Modelado del problema como búsqueda en espacio de estados

Formular el mapa del hospital como un problema de búsqueda definiendo de forma explícita:

- **Representación del estado**: cada estado corresponde a una posición del mapa del hospital, representada como una tupla `(fila, columna)`.
- **Estado inicial**: posición de partida, normalmente un pasillo cercano a una puerta o servicio.
- **Estados objetivo**: posiciones destino, definidas como pasillos cercanos a determinadas puertas o servicios.
- **Operadores / acciones**: movimientos permitidos en el mapa (arriba, abajo, izquierda, derecha).
- **Restricciones**:
  - no se permite salir de los límites del mapa,
  - no se pueden atravesar celdas no transitables o con penalización prohibitiva.
- **Función de coste**: cada movimiento tiene un coste asociado en función del tipo de celda destino, utilizando las penalizaciones definidas para las distintas áreas del hospital.
- **Heurística (cuando procede)**: estimación de la distancia restante hasta el objetivo (por ejemplo, distancia Manhattan), utilizada por los algoritmos informados.

---

### 2. Implementación y uso de algoritmos de búsqueda

Aplicar diferentes estrategias de búsqueda sobre el problema de planificación de rutas:

- **Búsqueda en Anchura (BFS)**, como algoritmo no informado de referencia.
- **Búsqueda Voraz**, guiada únicamente por la heurística.
- **Búsqueda A\***, que combina el coste acumulado y la heurística.

Para cada algoritmo se pretende:
- encontrar una ruta solución si existe,
- reconstruir el camino desde el estado inicial hasta el objetivo,
- calcular el coste total del recorrido.

---

### 3. Evaluación mediante métricas comparativas

Analizar y comparar el comportamiento de los algoritmos empleando métricas objetivas, tales como:

- **Coste total** del camino encontrado.
- **Número de pasos** o longitud del camino.
- **Número de nodos expandidos**, como medida del esfuerzo de búsqueda.
- **Tamaño máximo de la frontera**, como aproximación al consumo de memoria.
- **Tiempo de ejecución** aproximado.

El objetivo es poder justificar, a partir de los resultados, las ventajas e inconvenientes de cada algoritmo en este contexto.

---

### 4. Resolución de los ejercicios propuestos

#### Ejercicio 1: Ruta entre dos servicios
Calcular una ruta entre:
- el pasillo más cercano a un servicio de origen (por ejemplo, `D22`),
- y el pasillo más cercano a un servicio de destino (por ejemplo, `D1`).

El problema se resuelve con los distintos algoritmos y se comparan los resultados obtenidos.

#### Ejercicio 2: Planificación por tramos
Planificar la ruta de un responsable que debe:
- partir de una puerta principal del hospital,
- visitar una secuencia de servicios en un orden determinado,
- resolviendo el recorrido como la concatenación de varios tramos consecutivos.

Se busca:
- concatenar correctamente los caminos de cada tramo,
- acumular el coste total y las métricas globales,
- comparar el rendimiento de los algoritmos en recorridos más largos y complejos.

---

### 5. Presentación y análisis de resultados

Presentar los resultados de forma clara mediante:
- tablas comparativas para cada ejercicio,
- visualización de los recorridos sobre el mapa del hospital (cuando sea posible),
- conclusiones razonadas basadas en las métricas obtenidas y el comportamiento observado de cada algoritmo.


In [1]:
# IMPORTS PARA FUNCIONAMIENTO DEL PROYECTO
# =========================
# Imports estándar
# =========================
import pandas as pd

# =========================
# Imports del proyecto
# =========================
from utils.clases.mapa_hospital import mapa_hospital, penalizaciones
from utils.clases.clasesVisualizacion import VisualizadorHospital, SolucionesPosiciones

from utils.clases.ProblemaRuta import ProblemaRuta
from utils.clases.ResultadoBusqueda import ResultadoBusqueda
from utils.clases.MetricasBusqueda import MetricasBusqueda
from utils.clases.PlanificadorTramos import PlanificadorTramos

from utils.algoritmos.anchura import busqueda_anchura
from utils.algoritmos.voraz import busqueda_voraz
from utils.algoritmos.a_estrella import busqueda_a_estrella


## Ejercicio 1: Ruta desde D22 hasta D1

### ¿Qué vamos a hacer?
En el mapa del hospital, las puertas/servicios (por ejemplo `D22` o `D1`) están representadas como celdas con su etiqueta.
Sin embargo, una ruta realista no termina “dentro” de la puerta, sino en un **pasillo accesible**.

Por eso, para cada puerta:
1. Localizamos la(s) celda(s) con la etiqueta (ej. `D22`).
2. Buscamos los pasillos `P` **adyacentes** (vecindad 4: arriba/abajo/izq/der).
3. Elegimos un pasillo representativo (determinista) como punto de inicio/fin.

Con esos dos puntos (pasillo cercano a `D22` y pasillo cercano a `D1`) definimos el problema de búsqueda:
- **Estado**: posición `(fila, columna)`
- **Acciones**: movimientos N/S/E/O sobre el mapa
- **Coste**: penalización de la celda destino (según el diccionario `penalizaciones`)
- **Heurística**: distancia Manhattan al objetivo (para Voraz y A*)

Finalmente, resolvemos el mismo problema con:
- Búsqueda en Anchura (BFS)
- Búsqueda Voraz
- Búsqueda A*
y comparamos resultados en una tabla final.


## 1) Helpers para elegir pasillo cercano

In [2]:
def vecinos4(pos):
    f, c = pos
    return [(f-1, c), (f+1, c), (f, c-1), (f, c+1)]

def localizar_celdas(etiqueta, mapa):
    celdas = []
    for f in range(len(mapa)):
        for c in range(len(mapa[0])):
            if mapa[f][c] == etiqueta:
                celdas.append((f, c))
    return celdas

def pasillos_adyacentes_a(etiqueta, mapa):
    puertas = localizar_celdas(etiqueta, mapa)
    pasillos = set()
    for p in puertas:
        for v in vecinos4(p):
            vf, vc = v
            if 0 <= vf < len(mapa) and 0 <= vc < len(mapa[0]):
                if mapa[vf][vc] == "P":
                    pasillos.add(v)
    return sorted(list(pasillos))

def pasillo_representativo(etiqueta, mapa):
    """
    Devuelve un pasillo 'P' asociado a una etiqueta (determinista).
    Si hay varios, escoge el primero (ordenado) para que el resultado sea reproducible.
    """
    ps = pasillos_adyacentes_a(etiqueta, mapa)
    if len(ps) == 0:
        raise ValueError(f"No se han encontrado pasillos adyacentes a la etiqueta {etiqueta}")
    return ps[0]


## 2) Definir inicio y objetivo del Ejercicio 1

In [3]:
inicio_ej1 = pasillo_representativo("D22", mapa_hospital)
objetivo_ej1 = pasillo_representativo("D1", mapa_hospital)

print("Inicio (pasillo cercano a D22):", inicio_ej1)
print("Objetivo (pasillo cercano a D1):", objetivo_ej1)


Inicio (pasillo cercano a D22): (32, 64)
Objetivo (pasillo cercano a D1): (3, 2)


## 3) Ejecutar ANCHURA (BFS)

In [4]:
problema_ej1 = ProblemaRuta(inicio_ej1, objetivo_ej1, mapa_hospital, penalizaciones)

res_bfs = busqueda_anchura(problema_ej1)

print("=== ANCHURA (BFS) ===")
print("Solución:", res_bfs.hay_solucion())
print("Coste:", res_bfs.get_coste())
print("Pasos:", len(res_bfs.get_camino()) - 1)
print("Nodos expandidos:", res_bfs.get_nodos_expandidos())
print("Frontera máx:", res_bfs.get_frontera_max())
print("Tiempo (ms):", res_bfs.get_tiempo_ms())


=== ANCHURA (BFS) ===
Solución: True
Coste: 8087
Pasos: 91
Nodos expandidos: 1329
Frontera máx: 36
Tiempo (ms): 8.595943450927734


## 4) Ejecutar VORAZ 

In [5]:
problema_ej1 = ProblemaRuta(inicio_ej1, objetivo_ej1, mapa_hospital, penalizaciones)

res_voraz = busqueda_voraz(problema_ej1)

print("=== VORAZ ===")
print("Solución:", res_voraz.hay_solucion())
print("Coste:", res_voraz.get_coste())
print("Pasos:", len(res_voraz.get_camino()) - 1)
print("Nodos expandidos:", res_voraz.get_nodos_expandidos())
print("Frontera máx:", res_voraz.get_frontera_max())
print("Tiempo (ms):", res_voraz.get_tiempo_ms())


=== VORAZ ===
Solución: True
Coste: 8087
Pasos: 91
Nodos expandidos: 123
Frontera máx: 41
Tiempo (ms): 1.0561943054199219


## 5) Ejecutar A

In [6]:
problema_ej1 = ProblemaRuta(inicio_ej1, objetivo_ej1, mapa_hospital, penalizaciones)

res_astar = busqueda_a_estrella(problema_ej1)

print("=== A* ===")
print("Solución:", res_astar.hay_solucion())
print("Coste:", res_astar.get_coste())
print("Pasos:", len(res_astar.get_camino()) - 1)
print("Nodos expandidos:", res_astar.get_nodos_expandidos())
print("Frontera máx:", res_astar.get_frontera_max())
print("Tiempo (ms):", res_astar.get_tiempo_ms())


=== A* ===
Solución: True
Coste: 91
Pasos: 91
Nodos expandidos: 258
Frontera máx: 70
Tiempo (ms): 2.8867721557617188


## 6) Tabla comparativa final

In [7]:

def fila_resumen(nombre, res):
    return {
        "Algoritmo": nombre,
        "Hay solución": res.hay_solucion(),
        "Coste": res.get_coste() if res.hay_solucion() else None,
        "Pasos": (len(res.get_camino()) - 1) if res.hay_solucion() else None,
        "Nodos expandidos": res.get_nodos_expandidos(),
        "Frontera máx": res.get_frontera_max(),
        "Tiempo (ms)": res.get_tiempo_ms(),
    }

df_ej1 = pd.DataFrame([
    fila_resumen("Anchura (BFS)", res_bfs),
    fila_resumen("Voraz", res_voraz),
    fila_resumen("A*", res_astar),
])

df_ej1


,Algoritmo,Hay solución,Coste,Pasos,Nodos expandidos,Frontera máx,Tiempo (ms)
0,Anchura (BFS),True,8087,91,1329,36,8.595943
1,Voraz,True,8087,91,123,41,1.056194
2,A*,True,91,91,258,70,2.886772


En la Tabla del Ejercicio 1 se observa que BFS y Voraz obtienen la misma longitud de camino (91 pasos) y un coste elevado (8087), lo que indica que ambos encuentran una ruta corta en número de movimientos pero que atraviesa zonas con alta penalización. En contraste, A* mantiene la misma longitud del camino, pero reduce el coste a 91, evidenciando que prioriza trayectos por pasillos y evita celdas penalizadas al incorporar el coste acumulado en la función de evaluación. En términos de eficiencia, BFS expande muchos más nodos al no estar guiado, mientras que Voraz minimiza expansiones gracias a la heurística. A* presenta un equilibrio: expande más que Voraz para garantizar una solución de menor coste, pero sigue siendo significativamente más eficiente que BFS.

## Comprobaciones finales

1) Confirmar que inicio y objetivo son PASILLOS

In [8]:
print("Inicio Ej1:", inicio_ej1, "tipo:", mapa_hospital[inicio_ej1[0]][inicio_ej1[1]])
print("Objetivo Ej1:", objetivo_ej1, "tipo:", mapa_hospital[objetivo_ej1[0]][objetivo_ej1[1]])


Inicio Ej1: (32, 64) tipo: P
Objetivo Ej1: (3, 2) tipo: P


2) Confirmar que están realmente “cerca” de D22 y D1

In [9]:
def adyacente_a_etiqueta(pasillo, etiqueta, mapa):
    for v in vecinos4(pasillo):
        f, c = v
        if 0 <= f < len(mapa) and 0 <= c < len(mapa[0]):
            if mapa[f][c] == etiqueta:
                return True
    return False

print("Inicio adyacente a D22:", adyacente_a_etiqueta(inicio_ej1, "D22", mapa_hospital))
print("Objetivo adyacente a D1:", adyacente_a_etiqueta(objetivo_ej1, "D1", mapa_hospital))


Inicio adyacente a D22: True
Objetivo adyacente a D1: True


3) Confirmar que el camino de A* es válido

In [10]:
camino1 = res_astar.get_camino()

def es_movimiento_valido(a, b):
    return abs(a[0]-b[0]) + abs(a[1]-b[1]) == 1

ok = True
for i in range(len(camino1)-1):
    if not es_movimiento_valido(camino1[i], camino1[i+1]):
        ok = False
        print("Movimiento inválido entre:", camino1[i], "->", camino1[i+1])
        break

print("Camino con movimientos válidos (4-dir):", ok)


Camino con movimientos válidos (4-dir): True


4) Confirmar que no pisa celdas prohibidas / raras

In [11]:
tipos = [mapa_hospital[f][c] for (f,c) in camino1]
print("Tipos de celda en el camino (set):", sorted(set(tipos)))


Tipos de celda en el camino (set): ['P']


In [12]:
print("¿Pisa alguna M?:", "M" in tipos)


¿Pisa alguna M?: False


5. Confirmar ruta optima

In [13]:
print(f"Ruta completa Ej1 desde {camino1[0]} hasta {camino1[-1]}")
print(f"Nº de pasos: {len(camino1) - 1}")
print("-" * 40)

for pos in camino1:
    print(pos)


Ruta completa Ej1 desde (32, 64) hasta (3, 2)
Nº de pasos: 91
----------------------------------------
(32, 64)
(31, 64)
(31, 63)
(31, 62)
(31, 61)
(31, 60)
(31, 59)
(31, 58)
(31, 57)
(31, 56)
(31, 55)
(31, 54)
(31, 53)
(31, 52)
(31, 51)
(30, 51)
(29, 51)
(28, 51)
(27, 51)
(26, 51)
(25, 51)
(24, 51)
(23, 51)
(22, 51)
(21, 51)
(21, 50)
(21, 49)
(21, 48)
(21, 47)
(21, 46)
(21, 45)
(21, 44)
(21, 43)
(21, 42)
(21, 41)
(21, 40)
(21, 39)
(21, 38)
(21, 37)
(21, 36)
(21, 35)
(20, 35)
(19, 35)
(18, 35)
(17, 35)
(16, 35)
(15, 35)
(14, 35)
(13, 35)
(12, 35)
(11, 35)
(10, 35)
(10, 34)
(10, 33)
(9, 33)
(8, 33)
(7, 33)
(6, 33)
(5, 33)
(4, 33)
(4, 32)
(4, 31)
(4, 30)
(4, 29)
(4, 28)
(4, 27)
(4, 26)
(4, 25)
(4, 24)
(4, 23)
(4, 22)
(4, 21)
(4, 20)
(4, 19)
(4, 18)
(4, 17)
(4, 16)
(4, 15)
(4, 14)
(4, 13)
(4, 12)
(4, 11)
(4, 10)
(4, 9)
(4, 8)
(4, 7)
(4, 6)
(4, 5)
(4, 4)
(4, 3)
(4, 2)
(3, 2)


## Visualizador del camino

In [ ]:
import os
sol = SolucionesPosiciones(camino1_vis, mapa_hospital)
visor = VisualizadorHospital(sol, mapa_hospital)
visor.mostrar()


## Ejercicio 2: Ruta del responsable de seguridad (paso por puertas en orden)

En este ejercicio el responsable de seguridad debe comprobar que las puertas de los distintos servicios están cerradas.
La ruta:

- **comienza en la puerta principal**, y
- debe pasar por las puertas **en el orden indicado** por el enunciado.

### Criterio de "pasar por una puerta"
Para considerar que el responsable “ha pasado por una puerta”, **no necesita entrar en la celda de la puerta**:
basta con situarse en la **celda de pasillo (`P`) más cercana** a dicha puerta (adyacente en vecindad 4).

### Enfoque de resolución
El recorrido completo se resuelve como una **planificación por tramos**:

- Tramo 1: puerta principal → Información
- Tramo 2: Información → Farmacia
- Tramo 3: Farmacia → Laboratorio
- ...
- Último tramo: Baño 2 → Baño 3

Cada tramo se resuelve como un problema de ruta estándar y el camino final se obtiene concatenando los tramos,
evitando duplicar la celda de unión entre tramos.


# 2) Definir puerta principal + orden de servicios
## 2.1 Orden de visita

In [14]:
orden_servicios = [
    "I",    # Información
    "F",    # Farmacia
    "LB",   # Laboratorio
    "AP",   # Anatomía Patológica
    "RX",   # Radiología
    "UCI1",
    "UCI2",
    "Q1",   # Quirófano 1
    "Q2",   # Quirófano 2
    "C1",   # Consulta 1
    "C2",
    "C3",
    "C4",
    "C5",
    "UE1",  # Unidad de Enfermería 1
    "UE2",  # Unidad de Enfermería 2
    "B1",   # Baño 1
    "B2",
    "B3"
]


## 2.2) Puerta principal
Como el enunciado no da coordenadas, lo hacemos reproducible:

elegimos un pasillo P del borde (representa entrada/salida),

el más cercano a los pasillos adyacentes a Información (I) para que tenga sentido.

In [15]:
# Puerta principal: pasillo 'P' del borde más cercano a Información (I)
filas = len(mapa_hospital)
cols = len(mapa_hospital[0])

pasillos_borde = []
for f in range(filas):
    for c in range(cols):
        if mapa_hospital[f][c] == "P" and (f == 0 or f == filas-1 or c == 0 or c == cols-1):
            pasillos_borde.append((f, c))

pasillos_info = pasillos_adyacentes_a("I", mapa_hospital)

puerta_principal = None
mejor_dist = 10**9

for p in pasillos_borde:
    for i in pasillos_info:
        d = abs(p[0] - i[0]) + abs(p[1] - i[1])
        if d < mejor_dist:
            mejor_dist = d
            puerta_principal = p

print("Puerta principal:", puerta_principal, "| distancia a I:", mejor_dist)


Puerta principal: (17, 0) | distancia a I: 2


## 3) Convertir servicios a “pasillo más cercano”

In [16]:
objetivos_ej2 = [pasillo_representativo(s, mapa_hospital) for s in orden_servicios]

print("Número de paradas:", len(objetivos_ej2))
print("Primeras paradas:", list(zip(orden_servicios[:5], objetivos_ej2[:5])))


Número de paradas: 19
Primeras paradas: [('I', (17, 2)), ('F', (15, 49)), ('LB', (10, 12)), ('AP', (25, 12)), ('RX', (12, 30))]


## 4) Ejecutar por separado (BFS, Voraz, A*) usando planificación por tramos

In [17]:
planificador = PlanificadorTramos(mapa_hospital, penalizaciones)


### 4.1 Anchura (BFS)

In [18]:
res2_bfs = planificador.planificar(puerta_principal, objetivos_ej2, busqueda_anchura)

print("=== EJ2 - ANCHURA (BFS) ===")
print("Solución:", res2_bfs.hay_solucion())
print("Coste total:", res2_bfs.get_coste())
print("Pasos:", len(res2_bfs.get_camino()) - 1)
print("Nodos expandidos:", res2_bfs.get_nodos_expandidos())
print("Frontera máx:", res2_bfs.get_frontera_max())
print("Tiempo (ms):", res2_bfs.get_tiempo_ms())


=== EJ2 - ANCHURA (BFS) ===
Solución: True
Coste total: 74466
Pasos: 502
Nodos expandidos: 8239
Frontera máx: 43
Tiempo (ms): 57.531118392944336


### 4.2 Voraz

In [19]:
res2_voraz = planificador.planificar(puerta_principal, objetivos_ej2, busqueda_voraz)

print("=== EJ2 - VORAZ ===")
print("Solución:", res2_voraz.hay_solucion())
print("Coste total:", res2_voraz.get_coste())
print("Pasos:", len(res2_voraz.get_camino()) - 1)
print("Nodos expandidos:", res2_voraz.get_nodos_expandidos())
print("Frontera máx:", res2_voraz.get_frontera_max())
print("Tiempo (ms):", res2_voraz.get_tiempo_ms())


=== EJ2 - VORAZ ===
Solución: True
Coste total: 87480
Pasos: 522
Nodos expandidos: 896
Frontera máx: 35
Tiempo (ms): 5.921125411987305


### 4.3 A*

In [20]:
res2_astar = planificador.planificar(puerta_principal, objetivos_ej2, busqueda_a_estrella)

print("=== EJ2 - A* ===")
print("Solución:", res2_astar.hay_solucion())
print("Coste total:", res2_astar.get_coste())
print("Pasos:", len(res2_astar.get_camino()) - 1)
print("Nodos expandidos:", res2_astar.get_nodos_expandidos())
print("Frontera máx:", res2_astar.get_frontera_max())
print("Tiempo (ms):", res2_astar.get_tiempo_ms())


=== EJ2 - A* ===
Solución: True
Coste total: 594
Pasos: 594
Nodos expandidos: 1162
Frontera máx: 44
Tiempo (ms): 8.713245391845703


## 5) Tabla comparativa final

In [21]:

def fila_resumen(nombre, res):
    return {
        "Algoritmo": nombre,
        "Hay solución": res.hay_solucion(),
        "Coste total": res.get_coste() if res.hay_solucion() else None,
        "Pasos": (len(res.get_camino()) - 1) if res.hay_solucion() else None,
        "Nodos expandidos": res.get_nodos_expandidos(),
        "Frontera máx": res.get_frontera_max(),
        "Tiempo (ms)": res.get_tiempo_ms(),
    }

df_ej2 = pd.DataFrame([
    fila_resumen("Anchura (BFS)", res2_bfs),
    fila_resumen("Voraz", res2_voraz),
    fila_resumen("A*", res2_astar),
])

df_ej2


,Algoritmo,Hay solución,Coste total,Pasos,Nodos expandidos,Frontera máx,Tiempo (ms)
0,Anchura (BFS),True,74466,502,8239,43,57.531118
1,Voraz,True,87480,522,896,35,5.921125
2,A*,True,594,594,1162,44,8.713245


En el Ejercicio 2, al tratarse de una planificación multiobjetivo resuelta por tramos, las diferencias entre algoritmos se intensifican. La búsqueda en anchura completa la ruta pero requiere un esfuerzo computacional muy elevado (8239 nodos expandidos y 57.5 ms), debido a la ausencia de heurística y a la repetición de búsquedas en cada tramo. La búsqueda voraz reduce drásticamente el tiempo y los nodos expandidos al guiarse únicamente por la heurística, pero al ignorar el coste acumulado atraviesa con frecuencia zonas penalizadas, obteniendo el mayor coste total (87480). Por el contrario, A* combina el coste real con la heurística, logrando minimizar el coste total del recorrido (594), lo que indica que prioriza rutas por pasillos y evita zonas de alto coste; todo ello con un esfuerzo computacional moderado, superior al voraz pero muy inferior a BFS.

# Comprobaciones finales

Comprobar que inicio y objetivo son correctos

In [22]:
print("Tipo inicio:", mapa_hospital[inicio_ej1[0]][inicio_ej1[1]])
print("Tipo objetivo:", mapa_hospital[objetivo_ej1[0]][objetivo_ej1[1]])


Tipo inicio: P
Tipo objetivo: P


Comprobar que pasa por TODOS los objetivos

In [23]:
camino2 = set(res2_astar.get_camino())
faltan = [(s, obj) for s, obj in zip(orden_servicios, objetivos_ej2) if obj not in camino2]
print("Puertas NO visitadas:", faltan)


Puertas NO visitadas: []


In [28]:
camino = set(res_astar.get_camino())
faltan = [(s, obj) for s, obj in zip(orden_servicios, objetivos_ej2) if obj not in camino]
print("Puertas visitadas:", faltan)


Puertas visitadas: [('I', (17, 2)), ('F', (15, 49)), ('LB', (10, 12)), ('AP', (25, 12)), ('RX', (12, 30)), ('UCI1', (10, 37)), ('UCI2', (23, 50)), ('Q1', (12, 53)), ('Q2', (23, 53)), ('C1', (13, 17)), ('C2', (13, 19)), ('C3', (17, 19)), ('C4', (21, 17)), ('C5', (22, 19)), ('UE2', (31, 33)), ('B1', (3, 36)), ('B2', (9, 63)), ('B3', (26, 63))]


## Visor de la ruta

In [48]:

solucion_ej2 = SolucionesPosiciones(
    camino=res_ej2.get_camino(),
    mapa_hospital=mapa_hospital
)

visor = VisualizadorHospital(solucion_ej2, mapa_hospital)

visor.mostrar()
